In [1]:
import pathlib
import os
from pathlib import Path
import sys
 
if sys.argv:
    sys.path.insert(0, str(pathlib.Path(os.path.dirname(os.path.abspath(""))).resolve()))
%load_ext autoreload
%autoreload 2

In [2]:
from src.utils import get_paths_train_test, train_test_model
from src.factory import DESCNUpliftModelFactory

/home/ogrobertino/test_env/lib/python3.10/site-packages/xgboost/core.py:377: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc >= 2.28) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(
Failed to import duecredit due to No module named 'duecredit'


In [3]:
DESCNUpliftModelFactory

src.factory.DESCNUpliftModelFactory

In [4]:
from src.models.NNUpliftModeling.DESCNUpliftModel import DESCNUpliftModel

In [5]:
ds_name = 'lazada_v2'
features_percent = 100
train_path, test_path = get_paths_train_test(ds_name=ds_name, features_percent=features_percent)

In [6]:
import pandas as pd
ds = pd.read_csv(train_path, sep='\t')

In [7]:
input_size = len(ds.columns) - 2
input_size

83

In [26]:
params = {
    'input_dim': input_size,             # Должно быть задано в соответствии с данными
    'share_dim': [32, 64], # Варианты размерности общих слоев
    'base_dim': [32, 64],   # Варианты размерности базовых слоев
    'do_rate': [0.1, 0.2, 0.3],  # Варианты dropout
    'batch_norm': [True, False], # Использование BatchNorm
    'normalization': ['none', 'divide'], # Тип нормализации
    'factual_loss_weight': [0.8, 1.0, 1.2], # Вес фактической потери
    'propensity_loss_weight': [0.05, 0.1, 0.2], # Вес потери пропенсити
    'tau_loss_weight': [0.05, 0.1, 0.2],    # Вес потери tau (если применимо)
    'batch_size': 32,
    'gradient_accumulation_steps' : 2,
    'epochs': 2
}

In [27]:
configs = DESCNUpliftModel.generate_config(count=4, params = params)

In [28]:
factory = DESCNUpliftModelFactory
config = configs[0]
batch_size=32
max_size=100000

In [29]:
model, train, test = factory.create(config, train_path, test_path)

/home/ogrobertino/test_env/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [12]:
# from torch.utils.data import DataLoader
# dl = DataLoader(
#     train, 
#     batch_size=32, 
#     shuffle=True,

#     num_workers=0
# )

In [31]:
train_test_model(ds_name='lazada_v2', features_percent=100, factory=DESCNUpliftModelFactory, config=configs[0])   

/home/ogrobertino/test_env/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/2 - 20.0% - Loss: 0.1114
Epoch 1/2 - 40.0% - Loss: 0.1051
Epoch 1/2 - 60.0% - Loss: 0.1019
Epoch 1/2 - 80.0% - Loss: 0.1004
Epoch 1/2 - 100.0% - Loss: 0.0993
Validation


UnboundLocalError: local variable 'uplift_scores' referenced before assignment

In [ ]:
# for i in tqdm(range(len(configs)), desc="Training models"):
#     train_test_model(ds_name='lazada', features_percent=100, factory=DRModelFactory, config=configs[i])   